In [1]:
import pickle
import pandas as pd
import numpy as np
from pyspark.sql import DataFrame
from aeon.classification.distance_based import ProximityTree, ProximityForest
import logging

from pyspark.sql import SparkSession
import os
from pyspark.sql import SparkSession
from data_ingestion import DataIngestion
from preprocessing import Preprocessor
from prediction_manager import PredictionManager
from local_model_manager import LocalModelManager
from evaluation import Evaluator
from utilities import show_compact
import time
import json

In [2]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("ProximityTree") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "..", ".."))

config_DataIngestion = {
            # "data_path": project_root + "/ECG5000/*.tsv" # IF WE DECIDE TO USE .tsv FILES
            "data_path": project_root + "/fulldataset_ECG5000.csv", # IF WE DECIDE TO USE .csv FILES
            "data_percentage": 0.5  #*100% Percentage of data to load for SW development
            }   

ingestion = DataIngestion(spark,config_DataIngestion)
df = ingestion.load_data()
print(df.show(5))
df.printSchema()

Data Path is c:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\BigDataProject_repo/fulldataset_ECG5000.csv and loading 50.0% of data ++++++++++++++++++++++

Data size is :2528

+-----+-----------+-----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+-------------+-----------+-----------+-----------+-----------+-----------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-

In [4]:
config_preproc = {}    
preprocessor = Preprocessor(config_preproc)

preprocessed_df = preprocessor.run_preprocessing(df)
print(preprocessed_df.show(5))

+------------------+------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----

In [5]:
df.count()

2528

In [11]:
num_partitions = df.rdd.getNumPartitions() # Number of partitions in the DataFrame
num_partitions

2

In [7]:
for partition in range(df.rdd.getNumPartitions()):
    print(f"Number of rows in partition {partition+1}: {len(df.rdd.glom().collect()[partition])}")
    

Number of rows in partition 1: 1337
Number of rows in partition 2: 1191


In [8]:
n_splitters = 5

fraction = n_splitters / df.count()  # Fraction of data to sample for each splitter
fraction

0.001977848101265823

In [9]:
splitters = df.rdd.glom().sample(fraction=0.25, withReplacement=False).collect()
splitters

[[Row(label=1, _c1=-0.11252183, _c2=-2.8272038, _c3=-3.7738969, _c4=-4.3497511, _c5=-4.376041, _c6=-3.4749863, _c7=-2.1814082, _c8=-1.8182865, _c9=-1.2505219, _c10=-0.47749208, _c11=-0.36380791, _c12=-0.49195659, _c13=-0.42185509, _c14=-0.30920086, _c15=-0.4959387, _c16=-0.34211867, _c17=-0.35533627, _c18=-0.36791303, _c19=-0.31650279, _c20=-0.41237405, _c21=-0.47167181, _c22=-0.41345783, _c23=-0.36461703, _c24=-0.44929829, _c25=-0.47141866, _c26=-0.42477658, _c27=-0.46251673, _c28=-0.55247236, _c29=-0.47537519, _c30=-0.6942, _c31=-0.7018681, _c32=-0.59381178, _c33=-0.66068415, _c34=-0.71383066, _c35=-0.76980688, _c36=-0.67228161, _c37=-0.65367605, _c38=-0.63940562, _c39=-0.55930228, _c40=-0.59167032, _c41=-0.49322332, _c42=-0.46305183, _c43=-0.30164382, _c44=-0.23273401, _c45=-0.12505488, _c46=-0.15394314, _c47=-0.024357404, _c48=-0.065608758, _c49=0.034999258, _c50=0.061935219, _c51=0.07119542, _c52=0.12392505, _c53=0.10312371, _c54=0.22522849, _c55=0.12868305, _c56=0.30248315, _c57=

In [13]:
from random import sample

def sample_from_partition(index, iterator):
    if index==0:
        partition_data = list(iterator)
        # Randomly sample 5 values (or fewer if the partition has less than 5 rows)
        return sample(partition_data, min(5, len(partition_data)))
    
# def sample_from_partition(index, iterator):
#     for index in range(num_partitions):
#         partition_data = list(iterator)
#         # Randomly sample 5 values (or fewer if the partition has less than 5 rows)
#         return sample(partition_data, min(5, len(partition_data)))
    
# Apply the function to sample from a specific partition
sampled_data = df.rdd.mapPartitionsWithIndex(sample_from_partition).collect()
print("Randomly sampled data from partition 1 and 2:", sampled_data)

len(sampled_data)  # Number of sampled rows

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 20.0 failed 1 times, most recent failure: Lost task 1.0 in stage 20.0 (TID 26) (BenAtkinson-Dell-Inspiron3505.Home executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\benat\miniconda3\envs\bigdata_env\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1247, in main
    process()
  File "C:\Users\benat\miniconda3\envs\bigdata_env\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "C:\Users\benat\miniconda3\envs\bigdata_env\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 272, in dump_stream
    iterator = iter(iterator)
               ^^^^^^^^^^^^^^
TypeError: 'NoneType' object is not iterable

